# Setup

In [1]:
import os
import re
from copy import deepcopy
from typing import List, Tuple, Union

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from PIL import Image

In [2]:
# GLOBALS

# define your path below
# DATASET_PATH = ... # Ania
# DATASET_PATH = ... # Witek
DATASET_PATH = 'dataset/'  # Jacek

# hyperparameters
BATCH_SIZE = 32
EPOCHS = 100
LEARNING_RATE = 1e-3
NUM_WORKERS = 1
TRAIN_SET_SIZE = 0.8

In [3]:
torch.manual_seed(42)
np.random.seed(42)

# Data

In [4]:
class StorageDataset(torch.utils.data.Dataset):
    def __init__(self, path: str) -> None:
        self.path = path
        self.dirs = [d for d in os.listdir(self.path) if os.path.isdir(os.path.join(self.path, d))]
        self.items = self._get_file_target_map()

    def _get_file_target_map(self) -> List[Tuple[str, Tuple[float, float]]]:
        result = []
        for dir in self.dirs:
            labels = pd.read_csv(f'{self.path}/{dir}.csv', header=None, index_col=0, names=['speed', 'turn'])
            for file_name in os.listdir(os.path.join(self.path, dir)):
                file_path = os.path.join(self.path, dir, file_name)
                photo_id = int(re.search(r'\d+', file_name).group())
                if photo_id in labels.index:
                    target = labels.loc[photo_id]
                    result.append((file_path, (target['speed'], target['turn'], photo_id)))
        return result

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        path, target = self.items[idx]
        img = torch.tensor(np.asarray(Image.open(path)), dtype=torch.float32)
        target = torch.tensor(target[:2], dtype=torch.float32)
        return img, target

In [5]:
def train_test_split(dataset: StorageDataset, train_size: float = 0.8, shuffle: bool = True):
    train_keys, test_keys = set(), set()
    train_items, test_items = list(), list()
    for item in dataset.items:
        key = item[1][2]
        if key in train_keys:
            train_items.append(item)
        elif key in test_keys:
            test_items.append(item)
        else:
            if np.random.random() < train_size:
                train_keys.add(key)
                train_items.append(item)
            else:
                test_keys.add(key)
                test_items.append(item)
    train, test = deepcopy(dataset), deepcopy(dataset)
    train.items, test.items = train_items, test_items
    return train, test

In [6]:
dataset = StorageDataset(DATASET_PATH)
train_dataset, test_dataset = train_test_split(dataset, train_size=TRAIN_SET_SIZE, shuffle=True)
train = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
test = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

# ResNet18

In [ ]:
class ResNet18(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.backend = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
        self.backend.fc = nn.Linear(in_features=512, out_features=2, bias=True)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.backend(x)
        x = torch.clamp(x, min=-1.0, max=1.0)
        return x

In [ ]:
resnet = ResNet18()
resnet.cuda()
resnet = torch.jit.script(resnet)

In [ ]:
optimizer = torch.optim.Adam(resnet.parameters(), lr=LEARNING_RATE)
criterion = nn.MSELoss()

In [ ]:
train_len = len(train_dataset)

dummy_input = torch.randn(1, 3, 224, 224, device='cuda')
os.makedirs('models', exist_ok=True)
best_loss = np.inf

for epoch in range(EPOCHS):
    print(f'epoch {epoch}')
    train_loss_history, test_loss_history = [], []

    resnet.train()
    for i, (x_train, y_train) in enumerate(train):
        if not i % 5:
            print(f'{i}/{int(train_len / BATCH_SIZE)}', end='\r')
        x_train = torch.movedim(x_train, -1, -3)
        x_train = x_train.to(torch.device('cuda'))
        y_train = y_train.to(torch.device('cuda'))
        optimizer.zero_grad()
        y_hat = resnet(x_train)
        loss = criterion(y_hat, y_train)
        loss.backward()
        optimizer.step()
        train_loss_history.append(float(loss.detach().cpu()))
    print(f'train loss: {sum(train_loss_history) / len(train_loss_history)}')

    resnet.eval()
    with torch.no_grad():
        for x_test, y_test in test:
            x_test = torch.movedim(x_test, -1, -3)
            x_test = x_test.to(torch.device('cuda'))
            y_test = y_test.to(torch.device('cuda'))
            y_hat = resnet(x_test)
            loss = criterion(y_hat, y_test)
            test_loss_history.append(float(loss.detach().cpu()))
    test_loss = sum(test_loss_history) / len(test_loss_history)
    if test_loss < best_loss:
        best_loss = test_loss
        torch.jit.save(resnet, f'models/resnet_{epoch}_{test_loss:.4f}.pt')
        torch.onnx.export(resnet, dummy_input, f'models/resnet_{epoch}_{test_loss:.4f}_opset_11.onnx', opset_version=11)
        torch.onnx.export(resnet, dummy_input, f'models/resnet_{epoch}_{test_loss:.4f}_opset_13.onnx', opset_version=13)
        print(f'test loss: {test_loss} (best, saving model)')
    else:
        print(f'test loss: {test_loss}')

# MobileNet

In [9]:
class MobileNetV2(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.backend = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
        self.backend.classifier[1] = nn.Linear(in_features=1280, out_features=2, bias=True)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.backend(x)
        x = torch.clamp(x, min=-1.0, max=1.0)
        return x

In [10]:
mobilenet = MobileNetV2()
mobilenet.cuda()
mobilenet = torch.jit.script(mobilenet)

Using cache found in /home/witold/.cache/torch/hub/pytorch_vision_v0.10.0


In [11]:
train_len = len(train_dataset)

dummy_input = torch.randn(1, 3, 224, 224, device='cuda')
os.makedirs('models', exist_ok=True)
best_loss = np.inf

for epoch in range(EPOCHS):
    print(f'epoch {epoch}')
    train_loss_history, test_loss_history = [], []

    mobilenet.train()
    for i, (x_train, y_train) in enumerate(train):
        if not i % 5:
            print(f'{i}/{int(train_len / BATCH_SIZE)}', end='\r')
        x_train = torch.movedim(x_train, -1, -3)
        x_train = x_train.to(torch.device('cuda'))
        y_train = y_train.to(torch.device('cuda'))
        optimizer.zero_grad()
        y_hat = mobilenet(x_train)
        loss = criterion(y_hat, y_train)
        loss.backward()
        optimizer.step()
        train_loss_history.append(float(loss.detach().cpu()))
        break
    print(f'train loss: {sum(train_loss_history) / len(train_loss_history)}')

    mobilenet.eval()
    with torch.no_grad():
        for x_test, y_test in test:
            x_test = torch.movedim(x_test, -1, -3)
            x_test = x_test.to(torch.device('cuda'))
            y_test = y_test.to(torch.device('cuda'))
            y_hat = mobilenet(x_test)
            loss = criterion(y_hat, y_test)
            test_loss_history.append(float(loss.detach().cpu()))
            break
    test_loss = sum(test_loss_history) / len(test_loss_history)
    if test_loss < best_loss:
        best_loss = test_loss
        torch.jit.save(mobilenet, f'models/resnet_{epoch}_{test_loss:.4f}.pt')
        torch.onnx.export(mobilenet, dummy_input, f'models/mobilenet_{epoch}_{test_loss:.4f}_opset_11.onnx', opset_version=11)
        torch.onnx.export(mobilenet, dummy_input, f'models/mobilenet_{epoch}_{test_loss:.4f}_opset_13.onnx', opset_version=13)
        print(f'test loss: {test_loss} (best, saving model)')
    else:
        print(f'test loss: {test_loss}')

In [12]:
train_len = len(train_dataset)

dummy_input = torch.randn(1, 3, 224, 224, device='cuda')
os.makedirs('models', exist_ok=True)
best_loss = np.inf

for epoch in range(EPOCHS):
    print(f'epoch {epoch}')
    train_loss_history, test_loss_history = [], []

    mobilenet.train()
    for i, (x_train, y_train) in enumerate(train):
        if not i % 5:
            print(f'{i}/{int(train_len / BATCH_SIZE)}', end='\r')
        x_train = torch.movedim(x_train, -1, -3)
        x_train = x_train.to(torch.device('cuda'))
        y_train = y_train.to(torch.device('cuda'))
        optimizer.zero_grad()
        y_hat = mobilenet(x_train)
        loss = criterion(y_hat, y_train)
        loss.backward()
        optimizer.step()
        train_loss_history.append(float(loss.detach().cpu()))
        break
    print(f'train loss: {sum(train_loss_history) / len(train_loss_history)}')

    mobilenet.eval()
    with torch.no_grad():
        for x_test, y_test in test:
            x_test = torch.movedim(x_test, -1, -3)
            x_test = x_test.to(torch.device('cuda'))
            y_test = y_test.to(torch.device('cuda'))
            y_hat = mobilenet(x_test)
            loss = criterion(y_hat, y_test)
            test_loss_history.append(float(loss.detach().cpu()))
            break
    test_loss = sum(test_loss_history) / len(test_loss_history)
    if test_loss < best_loss:
        best_loss = test_loss
        torch.jit.save(mobilenet, f'models/mobilenet_{epoch}_{test_loss:.4f}.pt')
        torch.onnx.export(mobilenet, dummy_input, f'models/mobilenet_{epoch}_{test_loss:.4f}_opset_11.onnx', opset_version=11)
        torch.onnx.export(mobilenet, dummy_input, f'models/mobilenet_{epoch}_{test_loss:.4f}_opset_13.onnx', opset_version=13)
        print(f'test loss: {test_loss} (best, saving model)')
    else:
        print(f'test loss: {test_loss}')

epoch 0
train loss: 0.4787295460700989
test loss: 0.5124832391738892 (best, saving model)
epoch 1
train loss: 0.3226928412914276
test loss: 0.42607539892196655 (best, saving model)
epoch 2
train loss: 0.3831251561641693
test loss: 0.28809764981269836 (best, saving model)
epoch 3
train loss: 0.27176299691200256
test loss: 0.8398078083992004
epoch 4
train loss: 0.4734652042388916
test loss: 0.6289067268371582
epoch 5
train loss: 0.4261825978755951
test loss: 0.7455623149871826
epoch 6
train loss: 0.323070228099823
test loss: 0.6035286784172058
epoch 7
train loss: 0.31416329741477966
test loss: 0.39479127526283264
epoch 8
train loss: 0.4514525532722473
test loss: 0.38549119234085083
epoch 9
train loss: 0.5408028364181519
test loss: 0.35558322072029114
epoch 10
train loss: 0.3321213126182556
test loss: 0.3801511228084564
epoch 11
train loss: 0.3627023696899414
test loss: 0.497458279132843
epoch 12
train loss: 0.2711392045021057
test loss: 0.3706629276275635
epoch 13
train loss: 0.435320198